## Install Libraries

In [1]:
# %pip install -q torchsummary
# %pip install -q torchgeometry
%pip install -q segmentation-models-pytorch
# %pip install -q timm

Note: you may need to restart the kernel to use updated packages.


## Import Libraries

In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
from torchvision.io import read_image
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, random_split, DataLoader, Subset
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import os

# import timm
import torch
import torch.nn as nn
import torch.optim as optim
import segmentation_models_pytorch as smp
import wandb

/opt/conda/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.17). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-5b1fe213-19c6-ef5f-4ad7-8e6b8d35fd00)


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## Read data

In [5]:
images_path = "/kaggle/input/bkai-igh-neopolyp/train/train/"
image_path = []
TRAIN_DIR = '/kaggle/input/bkai-igh-neopolyp/train/train'
for root, dirs, files in os.walk(TRAIN_DIR):
    for file in files:
        path = os.path.join(root,file)
        image_path.append(path)
        
len(image_path)

1000

In [6]:
# import os
# import cv2
# import numpy as np

# def calculate_mean_image_size(image_dir):
#     widths = []
#     heights = []

#     for filename in os.listdir(image_dir):
#         if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
#             img_path = os.path.join(image_dir, filename)
#             image = cv2.imread(img_path)
#             if image is not None:
#                 h, w = image.shape[:2]
#                 heights.append(h)
#                 widths.append(w)
#             else:
#                 print(f"Warning: Unable to read image {img_path}")

#     mean_width = int(np.mean(widths))
#     mean_height = int(np.mean(heights))
#     median_width = int(np.median(widths))
#     median_height = int(np.median(heights))

#     print(f"Number of images: {len(widths)}")
#     print(f"Mean Width: {mean_width}, Mean Height: {mean_height}")
#     print(f"Median Width: {median_width}, Median Height: {median_height}")

#     return mean_width, mean_height, widths, heights

# # Example usage:
# image_directory = '/kaggle/input/bkai-igh-neopolyp/test/test'
# mean_w, mean_h, widths, heights = calculate_mean_image_size(image_directory)

In [7]:
# import matplotlib.pyplot as plt

# def plot_dimension_distribution(widths, heights):
#     plt.figure(figsize=(12, 5))

#     plt.subplot(1, 2, 1)
#     plt.hist(widths, bins=20, color='skyblue')
#     plt.title('Width Distribution')
#     plt.xlabel('Width')
#     plt.ylabel('Frequency')

#     plt.subplot(1, 2, 2)
#     plt.hist(heights, bins=20, color='salmon')
#     plt.title('Height Distribution')
#     plt.xlabel('Height')
#     plt.ylabel('Frequency')

#     plt.tight_layout()
#     plt.show()

# # After collecting widths and heights in the function above:
# plot_dimension_distribution(widths, heights)

In [8]:
mask_path = []
TRAIN_MASK_DIR = '/kaggle/input/bkai-igh-neopolyp/train_gt/train_gt'
for root, dirs, files in os.walk(TRAIN_MASK_DIR):
    for file in files:
        path = os.path.join(root,file)
        mask_path.append(path)
        
len(mask_path)

1000

In [9]:
class TrainDataset(Dataset):
    def __init__(self, img_dir, label_dir, resize=None, transform=None):
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.resize = resize
        self.transform = transform
        self.images = os.listdir(self.img_dir)

    def __len__(self):
        return len(self.images)
    
    def read_mask(self, mask_path):
        image = cv2.imread(mask_path)
        image = cv2.resize(image, self.resize, interpolation=cv2.INTER_AREA)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        lower_red1 = np.array([0, 100, 20])
        upper_red1 = np.array([10, 255, 255])
        lower_red2 = np.array([160,100,20])
        upper_red2 = np.array([179,255,255])
        
        lower_mask_red = cv2.inRange(image, lower_red1, upper_red1)
        upper_mask_red = cv2.inRange(image, lower_red2, upper_red2)
        
        red_mask = lower_mask_red + upper_mask_red
        red_mask[red_mask != 0] = 1

        green_mask = cv2.inRange(image, (36, 25, 25), (70, 255, 255))
        green_mask[green_mask != 0] = 2

        full_mask = cv2.bitwise_or(red_mask, green_mask)
        full_mask = np.expand_dims(full_mask, axis=-1) 
        full_mask = full_mask.astype(np.uint8)
        
        return full_mask

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx])
        label_path = os.path.join(self.label_dir, self.images[idx])
        image = cv2.imread(img_path)  #  BGR
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Convert to RGB
        label = self.read_mask(label_path)  
        image = cv2.resize(image, self.resize, interpolation=cv2.INTER_AREA)
        if self.transform:
            image = self.transform(image)
            
        return image, label
    
class TestDataset(Dataset):
    def __init__(self, img_dir="path/to/data", resize = None, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.resize = resize
        self.images = os.listdir(self.img_dir)

    def __len__(self):
        return len(self.images)
    

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx])
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        height, width, channels = image.shape
        image = cv2.resize(image, self.resize, interpolation=cv2.INTER_AREA)
        name =  os.path.basename(img_path)
        if self.transform:
            transformed = self.transform(image=image)
            image = transformed["image"]
        return image, height, width, name[:-5]

### Hyperparameters

In [10]:
img_resize = (256, 256)
batch_size = 32
learning_rate = 1e-3
alpha = 0.5  # Weight of cross entropy loss 

In [11]:
dataset = TrainDataset(img_dir= TRAIN_DIR,
                             label_dir= TRAIN_MASK_DIR,
                             resize= img_resize,
                             transform = None)

## Augmentation

In [12]:
class AugmentDataset(Dataset):
    def __init__(self, dataset, transform=None, length_multiplier=1):
        self.dataset = dataset
        self.transform = transform
        self.length_multiplier = length_multiplier
        
    def __getitem__(self, idx):
        actual_idx = idx % len(self.dataset)
        image, label = self.dataset[actual_idx]
        if self.transform:
            transformed = self.transform(image=image, mask=label)
            image = transformed['image'].float()
            label = transformed['mask'].float()
            label = label.permute(2, 0, 1)
        return image, label

    def __len__(self):
        return len(self.dataset) * self.length_multiplier

In [13]:
class CustomDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset) * 4  # Four copies per image

    def __getitem__(self, idx):
        actual_idx = idx // 4
        flip_idx = idx % 4

        image, label = self.dataset[actual_idx]

        # Apply flips based on flip_idx
        if flip_idx == 1:
            image = cv2.flip(image, 1)  # Horizontal flip
            label = cv2.flip(label, 1)
        elif flip_idx == 2:
            image = cv2.flip(image, 0)  # Vertical flip
            label = cv2.flip(label, 0)
        elif flip_idx == 3:
            image = cv2.flip(image, -1)  # Horizontal and vertical flip
            label = cv2.flip(label, -1)

        if self.transform:
            transformed = self.transform(image=image, mask=label)
            image = transformed['image'].float()
            label = transformed['mask'].float()
            label = label.permute(2, 0, 1) # Add channel dimension to label

        return image, label

In [14]:
train_transformation = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomGamma (gamma_limit=(70, 130), always_apply=False, p=0.2),
    A.RGBShift(p=0.3, r_shift_limit=10, g_shift_limit=10, b_shift_limit=10),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

val_transformation = A.Compose([
    A.Normalize(mean=(0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])
# train_transformation = A.Compose([
#     A.HorizontalFlip(p=0.5),
#     A.VerticalFlip(p=0.5),
#     A.RandomGamma (gamma_limit=(70, 130), always_apply=False, p=0.2),
#     A.RGBShift(p=0.3, r_shift_limit=10, g_shift_limit=10, b_shift_limit=10),
#     A.OneOf([A.Blur(), A.GaussianBlur(), A.GlassBlur(), A.MotionBlur(), A.GaussNoise(), A.Sharpen(), A.MedianBlur(), A.MultiplicativeNoise()], p=0.35),
#     A.CoarseDropout(p=0.2, max_height=15, max_width=15, fill_value=255),
#     A.RandomSnow(snow_point_lower=0.1, snow_point_upper=0.15, brightness_coeff=1.5, p=0.09),
# #     A.RandomShadow(p=0.1),
#     A.ShiftScaleRotate(p=0.15, border_mode=cv2.BORDER_CONSTANT, shift_limit=0.15, scale_limit=0.15),
#     A.RandomCrop((img_resize, img_resize)),
#     A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
#     ToTensorV2(),
# ])
class UnNormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std
        
    def __call__(self, tensor):
        """
        Args:
            tensor (Tensor): Tensor image of size (C, H, W) to be normalized.
        Returns:
            Tensor: Normalized image.
        """
        for t, m, s in zip(tensor, self.mean, self.std):
            t.mul_(s).add_(m)
            # The normalize code -> t.sub_(m).div_(s)
        return tensor
    
unorm = UnNormalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))

In [15]:
# Define the split sizes
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# Randomly split the dataset
# generator = torch.Generator().manual_seed(7)
# train_dataset, val_dataset = random_split(dataset, [train_size, val_size], generator=generator)
train_dataset = Subset(dataset, range(train_size))
val_dataset = Subset(dataset, range(train_size, train_size + val_size))
train_dataset = AugmentDataset(train_dataset, transform=train_transformation, length_multiplier=1)
val_dataset = AugmentDataset(val_dataset, transform=val_transformation)
# train_dataset = CustomDataset(train_dataset, transform=train_transformation)
# val_dataset = CustomDataset(val_dataset, transform=val_transformation)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [16]:
test_dataset = TestDataset(img_dir="/kaggle/input/bkai-igh-neopolyp/test/test", resize=img_resize, transform=val_transformation)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [17]:
len(train_dataset), len(val_dataset), len(test_dataset)

(800, 200, 200)

In [18]:
# def visualize_samples(train_loader, val_loader, test_loader, unorm, num_samples=3):
#     def show_images(images, labels=None, title=""):
#         fig, axes = plt.subplots(1, len(images), figsize=(15, 5))
#         if len(images) == 1:
#             axes = [axes]
#         for i, img in enumerate(images):
#             img = unorm(img)
#             img = img.permute(1, 2, 0).cpu().numpy()
#             axes[i].imshow(img)
#             if labels is not None:
#                 axes[i].imshow(labels[i].squeeze().cpu().numpy(), alpha=0.3, cmap='jet')
#             axes[i].axis('off')
#         fig.suptitle(title)
#         plt.show()

#     # Visualize train samples
#     train_images, train_labels = next(iter(train_loader))
#     show_images(train_images[:num_samples], train_labels[:num_samples], title="Train Samples")

#     # Visualize validation samples
#     val_images, val_labels = next(iter(val_loader))
#     show_images(val_images[:num_samples], val_labels[:num_samples], title="Validation Samples")

#     # Visualize test samples
#     test_images, _, _, _ = next(iter(test_loader))
#     show_images(test_images[:num_samples], title="Test Samples")

# visualize_samples(train_loader, val_loader, test_loader, unorm)

## Define Model

In [19]:
# class DoubleConv(nn.Module):
#     def __init__(self, in_channels, out_channels):
#         super(DoubleConv, self).__init__()
#         self.double_conv = nn.Sequential(
#             nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
#             nn.BatchNorm2d(out_channels),
#             nn.LeakyReLU(inplace=True),
#             nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
#             nn.BatchNorm2d(out_channels),
#             nn.LeakyReLU(inplace=True),
#         )

#     def forward(self, x):
#         return self.double_conv(x)
    
    
# class DownBlock(nn.Module):
#     def __init__(self, in_channels, out_channels):
#         super(DownBlock, self).__init__()
#         self.double_conv = DoubleConv(in_channels, out_channels)
#         self.down_sample = nn.MaxPool2d(2)

#     def forward(self, x):
#         skip_out = self.double_conv(x)
#         down_out = self.down_sample(skip_out)
#         return (down_out, skip_out)

    
# class UpBlock(nn.Module):
#     def __init__(self, in_channels, out_channels, up_sample_mode):
#         super(UpBlock, self).__init__()
#         if up_sample_mode == 'conv_transpose':
#             if out_channels*4 == in_channels:
#                 self.up_sample = nn.ConvTranspose2d(in_channels-out_channels*2, in_channels-out_channels*2, kernel_size=2, stride=2) 
#             else:
#                 self.up_sample = nn.ConvTranspose2d(in_channels-out_channels, in_channels-out_channels, kernel_size=2, stride=2)
#         else:
#             self.up_sample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
#         self.double_conv = DoubleConv(in_channels, out_channels)

#     def forward(self, down_input, skip_input):
# #         print("down", down_input.shape)
# #         print("skip", skip_input.shape)
#         x = self.up_sample(down_input)
# #         print("x",x.shape)
#         x = torch.cat([x, skip_input], dim=1)
#         return self.double_conv(x)

In [20]:
# # ResUnet
# class PolypModel(nn.Module):
#     def __init__(self, out_classes=3, up_sample_mode='conv_transpose'):
#         super().__init__()
#         self.out_classes = out_classes
#         self.backbone = timm.create_model("resnet152", pretrained=True, features_only=True)
# #         self.down_conv1 = DownBlock(3, 64)
#         self.down_conv1 = DownBlock(64, 128)
#         self.down_conv2 = DownBlock(256, 512)
#         self.down_conv3 = DownBlock(512, 1024)
#         self.down_conv4 = DownBlock(1024, 2048)
#         self.up_sample_mode = up_sample_mode
#         self.block_neck = DoubleConv(2048, 1024)
#         self.block_up1 = UpBlock(1024+1024, 512,self.up_sample_mode)
#         self.block_up2 = UpBlock(512+512, 256,self.up_sample_mode)
#         self.block_up3 = UpBlock(256+256, 128,self.up_sample_mode)
#         self.block_up4 = UpBlock(128+64, 64,self.up_sample_mode)
#         self.conv_last = nn.Conv2d(64, out_classes, kernel_size=1)
#         self.upsample = nn.Upsample(scale_factor=2, mode="bilinear")
#         self.final_activation = nn.Softmax(dim=1)

#     def forward(self, x):
#         x1, x2, x3, x4, x5 = self.backbone(x)
#         x = self.block_neck(x5) # x (B, 1024, 8, 8)
#         x = self.block_up1(x, x4)
#         x = self.block_up2(x, x3)
#         x = self.block_up3(x, x2)
#         x = self.block_up4(x, x1 )
#         x = self.conv_last(x)
#         x = self.upsample(x)
#         x = self.final_activation(x)
#         return x

In [ ]:
# model = PolypModel()

import segmentation_models_pytorch as smp

model = smp.UnetPlusPlus(
    encoder_name="timm-resnest200e",        
    encoder_weights="imagenet",     
    in_channels=3,                  
    classes=3     
)

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.cache/torch/hub/checkpoints/resnet152-b121ed2d.pth
100%|██████████| 230M/230M [00:02<00:00, 90.4MB/s]


In [22]:
# checkpoint = torch.load('/kaggle/input/unetplusplus-resnet152/model.pth')
# model.load_state_dict(checkpoint['model'])
# model = model.to(device)

In [23]:
color_dict= {0: (0, 0, 0),
             1: (255, 0, 0),
             2: (0, 255, 0)}
def mask_to_rgb(mask, color_dict):
    output = np.zeros((mask.shape[0], mask.shape[1], 3))

    for k in color_dict.keys():
        output[mask==k] = color_dict[k]

    return np.uint8(output)    

In [24]:
# del train_dataset
# del val_dataset
# del test_dataset

## Train

Define Dice loss function

In [25]:
import wandb

wandb.login(
    # set the wandb project where this run will be logged
#     project= "PolypSegment", 
    key = "e02f7703b40a2b3e0ab4801d4cb1d86b3b7327a6",
)

wandb.init(
    project="PolypSegment",
    config={
        # "model" : "Unet",
        "init_learning_rate": learning_rate,
        "batch_size": batch_size,
        "optimizer": "Adam",
        "loss_functions": ["CrossEntropy", "Dice"],
        "alpha":alpha,
        "imsize": img_resize
    }
)

# Add the model to WandB
wandb.watch(model, log="all")

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tu7pham7 (hustcollab). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.3
wandb: Run data is saved locally in /kaggle/working/wandb/run-20241124_080923-hd38xsfw
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run deep-breeze-61
wandb: ⭐️ View project at https://wandb.ai/hustcollab/PolypSegment
wandb: 🚀 View run at https://wandb.ai/hustcollab/PolypSegment/runs/hd38xsfw


[]

In [ ]:
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, class_weights = None, gamma=2, ignore_index=None):
        super(FocalLoss, self).__init__()
        self.class_weights = class_weights
        self.gamma = gamma
        self.ignore_index = ignore_index

    def forward(self, inputs, targets):
        logpt = -F.cross_entropy(inputs, targets, reduction='none', ignore_index=self.ignore_index)
        pt = torch.exp(logpt)
        loss = self.class_weights * ((1 - pt) ** self.gamma) * logpt
        return loss.mean()

In [26]:
from tqdm import tqdm

model.to(device)
criterion_ce = nn.CrossEntropyLoss()
criterion_dice = smp.losses.DiceLoss(mode='multiclass')
best_val_loss = 999

Make use of learning rate scheduler

In [27]:
import torch.optim as optim
from torch.amp import autocast, GradScaler

scaler = GradScaler()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Initialize the scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    mode='min',                 # We want to minimize the validation loss
    factor=0.1,                 # Reduce LR by a factor of 0.5
    patience=0,                 # Number of epochs with no improvement after which LR will be reduced
    verbose=True,               # Print a message when LR is updated
    threshold=0.0001,           # Threshold for measuring the new optimum
    threshold_mode='rel',       # Mode for threshold
    cooldown=0,                 # Number of epochs to wait before resuming normal operation after LR has been reduced
    min_lr=1e-6                 # Lower bound on the learning rate
)

scheduler2 = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5)

/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [28]:
num_epochs = 20
epoch_bar = tqdm(total=num_epochs, desc='Total Progress')

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        labels = labels.squeeze(dim=1).long()
        with autocast(device_type="cuda"):
            outputs = model(images)
            loss_ce = criterion_ce(outputs, labels)
            loss_dice = criterion_dice(outputs, labels)
            loss = alpha * loss_ce + (1 - alpha) * loss_dice

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
        train_loss += loss.item()
        
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            labels = labels.squeeze(dim=1).long()
            
            with autocast(device_type="cuda"):
                outputs = model(images)
                loss_ce = criterion_ce(outputs, labels)
                loss_dice = criterion_dice(outputs, labels)
                loss = alpha * loss_ce + (1 - alpha) * loss_dice

            val_loss += loss.item()
            
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {val_loss/len(val_loader):.10f}")
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        checkpoint = { 
            'epoch': epoch,
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'loss': val_loss,
        }
        save_path = f'model.pth'
        torch.save(checkpoint, save_path)
        
    scheduler.step(val_loss)
    current_lr = optimizer.param_groups[0]['lr']
    epoch_bar.update(1)
    wandb.log({'Val_loss': val_loss/len(val_loader),'Train_loss': train_loss/len(train_loader), 'Learning_rate' : current_lr})
epoch_bar.close()

Total Progress:   0%|          | 0/20 [00:00<?, ?it/s]

Epoch [1/20], Loss: 3.2663187640


Total Progress:   5%|▌         | 1/20 [01:09<22:01, 69.56s/it]

Epoch [2/20], Loss: 0.3378334556


Total Progress:  10%|█         | 2/20 [02:09<19:12, 64.03s/it]

...

Epoch [17/20], Loss: 0.1819254969


Total Progress:  90%|█████████ | 18/20 [17:33<01:54, 57.21s/it]

Epoch [18/20], Loss: 0.1764565366
Epoch [19/20], Loss: 0.1586084004


Total Progress: 100%|██████████| 20/20 [19:30<00:00, 58.54s/it]

Epoch [20/20], Loss: 0.1809272873


In [29]:
# model = smp.UnetPlusPlus(
#     encoder_name="resnet34",        
#     # encoder_weights="imagenet",     
#     in_channels=3,                  
#     classes=3     
# )
# model = PolypModel()
# checkpoint = torch.load('/kaggle/input/model-polyp/colorization_model.pth')
# model.load_state_dict(checkpoint['model_state_dict'])
# model = model.to(device)
checkpoint = torch.load('/kaggle/working/model.pth')
model.load_state_dict(checkpoint['model'])
model = model.to(device)

/tmp/ipykernel_23/4093892069.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('/kaggle/working/model.pth')


In [30]:
model.eval()
val_loss = 0
with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)
        labels = labels.squeeze(dim=1).long()
        
        with autocast(device_type="cuda"):
            outputs = model(images)
            # loss_ce = criterion_ce(outputs, labels)
            loss_dice = criterion_dice(outputs, labels)
            # loss = alpha * loss_ce + (1 - alpha) * loss_dice
            print(loss_dice.item())
        val_loss += loss_dice.item()
print(f"Loss: {val_loss/len(val_loader):.10f}")

0.31018590927124023
0.3012106418609619
0.2889310419559479
0.25360262393951416
0.3469126224517822
0.31575465202331543
0.06808605045080185
Loss: 0.2692405060


In [31]:
# import shutil

# shutil.rmtree('prediction')

In [32]:
!mkdir prediction

## Get Testset Prediction

In [ ]:
import cv2
import numpy as np

def Smoothed_img(image):
    resized_image = image
    smoothed_image = cv2.GaussianBlur(resized_image, (15, 15), 0)
    red_pixels = (smoothed_image[:, :, 2] > 100)
    green_pixels = (smoothed_image[:, :, 1] > 100)
    black_pixels = ~(red_pixels | green_pixels)
    smoothed_image[red_pixels] = [0, 0, 255]
    smoothed_image[green_pixels] = [0, 255, 0]
    smoothed_image[black_pixels] = [0, 0, 0]

    # pixel_values = np.array(smoothed_image)
    # unique_values = np.unique(pixel_values)
    # print(unique_values)
    return smoothed_image

In [ ]:
model.eval()
with torch.no_grad():
    for image, h, w, name in test_loader:
        outputs = model(image.to(device))
        outputs = outputs.squeeze().cpu().numpy()
        outputs = np.argmax(outputs, axis=0)
        outputs = mask_to_rgb(outputs, color_dict)
        w = w.item()
        h = h.item()
        new_size = (w, h)
        resized_image = cv2.resize(outputs, new_size, interpolation=cv2.INTER_CUBIC)
        resized_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
        resized_image = Smoothed_img(resized_image)
        cv2.imwrite(f"prediction/{name[0]}.png", resized_image)
        

IndexError: boolean index did not match indexed array along dimension 2; dimension is 3 but corresponding boolean dimension is 256

In [ ]:
def rle_to_string(runs):
    return ' '.join(str(x) for x in runs)

def rle_encode_one_mask(mask):
    pixels = mask.flatten()
    pixels[pixels > 225] = 255
    pixels[pixels <= 225] = 0
    use_padding = False
    if pixels[0] or pixels[-1]:
        use_padding = True
        pixel_padded = np.zeros([len(pixels) + 2], dtype=pixels.dtype)
        pixel_padded[1:-1] = pixels
        pixels = pixel_padded
    rle = np.where(pixels[1:] != pixels[:-1])[0] + 2
    if use_padding:
        rle = rle - 1
    rle[1::2] = rle[1::2] - rle[:-1:2]
    
    return rle_to_string(rle)

def rle2mask(mask_rle, shape=(3,3)):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

def mask2string(dir):
    strings = []
    ids = []
    ws, hs = [[] for i in range(2)]
    for image_id in os.listdir(dir):
        id = image_id.split('.')[0]
        path = os.path.join(dir, image_id)
        # print(path)
        img = cv2.imread(path)[:,:,::-1]
        h, w = img.shape[0], img.shape[1]
        for channel in range(2):
            ws.append(w)
            hs.append(h)
            ids.append(f'{id}_{channel}')
            string = rle_encode_one_mask(img[:,:,channel])
            strings.append(string)
    r = {
        'ids': ids,
        'strings': strings,
    }
    return r


MASK_DIR_PATH = '/kaggle/working/prediction'
dir = MASK_DIR_PATH
res = mask2string(dir)
df = pd.DataFrame(columns=['Id', 'Expected'])
df['Id'] = res['ids']
df['Expected'] = res['strings']

df.to_csv(r'output.csv', index=False)

## Inference

In [ ]:
# !mkdir -p /kaggle/working/prediction

In [ ]:
# def inference(model, test_loader, device, color_dict, output_dir="inference"):
#     model.eval()
#     os.makedirs(output_dir, exist_ok=True)
#     with torch.no_grad():
#         for image, h, w, name in test_loader:
#             outputs = model(image.to(device))
#             outputs = outputs.squeeze()
#             outputs = np.argmax(outputs.cpu().numpy(), axis=0)
#             outputs = mask_to_rgb(outputs, color_dict)
#             w = w.item()
#             h = h.item()
#             new_size = (w, h)
#             resized_image = cv2.resize(outputs, new_size, interpolation=cv2.INTER_CUBIC)
#             resized_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
#             resized_image = Smoothed_img(resized_image)
#             cv2.imwrite(f"{output_dir}/{name[0]}.png", resized_image)

# inference(model, test_loader, device, color_dict)